In [9]:
import json
import requests
from bs4 import BeautifulSoup

In [10]:
with open('problemset.json') as f:
    data = json.load(f)

In [11]:
s = requests.Session()
final_data = {}
for j, i in enumerate(data['result']['problems']):
    prob_id = str(i['contestId']) + str(i['index'])
    problem_data = {}
    problem_data['name'] = i['name']
#     r = s.get('http://codeforces.com/problemset/problem/' + str(i['contestId']) + '/' + str(i['index']))
#     soup = BeautifulSoup(r.text)
#     print(soup.find_all("span", {"class" : "tag-box"}))
#     data = int(soup.find_all("span", {"class" : "tag-box"})[-1].text.strip()[1:])
#     problem_data['diffuiculty'] = data
    problem_data['tags'] = i['tags']
    if(prob_id not in final_data):
        final_data[prob_id] = {}
    final_data[prob_id] = problem_data

In [12]:
for i in range(1, 48):
    r = s.get('http://codeforces.com/problemset/page/' + str(i))
    soup = BeautifulSoup(r.text)
    for j in soup.find_all("div", {"class" : "datatable"}):
        for k in j.find_all("tr")[1:]:
            try:
                name = k.find_all("td", {"class" : "id"})[0].text.strip()
                difficulty = k.find_all("span", {"class" : "ProblemRating"})[0].text.strip()
            except:
                continue
            final_data[name]['difficulty'] = difficulty

1


/home/raghavan/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/raghavan/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [15]:
for i in final_data:
    if('difficulty' not in final_data[i]):
        final_data[i]['difficulty'] = None
        
with open('problems.json', 'w') as fp:
    json.dump(final_data, fp)